In [1]:
import pandas as pd
import os
from tqdm import tqdm
import yaml
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import csv
import time
import numpy as np

with open("/home/ubuntu/work/therapeutic_accelerator/config/main.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
with open("../config/keys.yaml", "r") as f:
    keys = yaml.load(f, Loader=yaml.FullLoader)
    
bucket_path = os.path.join(config['paths']['root'], config['paths']['mount'])

In [2]:
engine = create_engine(f'postgresql://postgres:{keys["postgres"]}@{config["database"]["host"]}:5432/postgres')

In [ ]:
# Create connection to postgres db
# from sqlalchemy.engine.url import URL

# postgres_db = {'drivername': 'postgres',
#                'username': 'postgres',
#                'password': keys["postgres"],
#                'host': config["database"]["host"],
#                'port': 5432}
# print(URL(**postgres_db))
# postgres = URL(**postgres_db)

In [ ]:
# # Create database metadata schema
# from sqlalchemy import create_engine
# from sqlalchemy import MetaData
# from sqlalchemy import Table
# from sqlalchemy import Column
# from sqlalchemy import Integer, String

# db_uri = 'sqlite:///db.sqlite'
# engine = create_engine(db_uri)

# # Create a metadata instance
# metadata = MetaData(engine)
# # Declare a table
# table = Table('Example',metadata,
#               Column('id',Integer, primary_key=True),
#               Column('name',String))
# # Create all tables
# metadata.create_all()
# for _t in metadata.tables:
#    print("Table: ", _t)

In [25]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table

# db_uri = 'sqlite:///db.sqlite'
# engine = create_engine(db_uri)

# Create a MetaData instance
metadata = MetaData()
print(metadata.tables)

# reflect db schema to MetaData
metadata.reflect(bind=engine)
print(metadata.tables)

metadata.tables.get('abstracts')

FacadeDict({})
FacadeDict({'attributes': Table('attributes', MetaData(), Column('index', BIGINT(), table=<attributes>), Column('corpusid', BIGINT(), table=<attributes>, primary_key=True, nullable=False), Column('externalids', TEXT(), table=<attributes>), Column('url', TEXT(), table=<attributes>), Column('title', TEXT(), table=<attributes>), Column('authors', TEXT(), table=<attributes>), Column('venue', TEXT(), table=<attributes>), Column('publicationvenueid', TEXT(), table=<attributes>), Column('year', DOUBLE_PRECISION(precision=53), table=<attributes>), Column('referencecount', BIGINT(), table=<attributes>), Column('citationcount', BIGINT(), table=<attributes>), Column('influentialcitationcount', BIGINT(), table=<attributes>), Column('isopenaccess', BOOLEAN(), table=<attributes>), Column('s2fieldsofstudy', TEXT(), table=<attributes>), Column('publicationtypes', TEXT(), table=<attributes>), Column('publicationdate', TEXT(), table=<attributes>), Column('journal', TEXT(), table=<attribut

Table('abstracts', MetaData(), Column('index', TEXT(), table=<abstracts>), Column('paperId', TEXT(), table=<abstracts>), Column('corpusId', TEXT(), table=<abstracts>), Column('abstract', TEXT(), table=<abstracts>), schema=None)

In [ ]:
# Rename column
# ALTER TABLE {table_name} RENAME COLUMN corpusid TO corpusId;

In [23]:
table_name = 'attributes'

# set primarykey column
query = f'''
    ALTER TABLE {table_name} ADD PRIMARY KEY (corpusId);
'''

# change type
with engine.connect() as conn: 
    conn.execute(query)

In [24]:
table_name = 'abstracts'

# set primarykey column
query = f'''
    ALTER TABLE {table_name} ADD PRIMARY KEY (corpusId);
'''

# change type
with engine.connect() as conn: 
    conn.execute(query)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "corpusid" of relation "abstracts" does not exist

[SQL: 
    ALTER TABLE abstracts ADD PRIMARY KEY (corpusId);
]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [2]:
# Query to pull in corpus IDs

sql = '''
    SELECT * FROM abstracts;
'''

# Context manager to open the connection to the database and execute the query. 
with engine.connect() as conn:
    query = conn.execute(text(sql))
    
# Get all records and store in dataframe
abstracts = pd.DataFrame(query.fetchall())

abstracts.head()


KeyboardInterrupt: 

In [6]:
# Query to pull in corpus IDs
sql = '''
    SELECT * FROM attributes;
'''

# Context manager to open the connection to the database and execute the query. 
with engine.connect() as conn:
    query = conn.execute(text(sql))
    
# Get all records and store in dataframe
attributes = pd.DataFrame(query.fetchall())
attributes.head()

,index,corpusid,externalids,url,title,authors,venue,publicationvenueid,year,referencecount,citationcount,influentialcitationcount,isopenaccess,s2fieldsofstudy,publicationtypes,publicationdate,journal,updated,id
0,0,52273203,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/39063bcd...,Large conformational changes of a highly dynam...,"{""{\""authorId\"": \""51285510\"", \""name\"": \""Isa...",Communications Biology,069e05e7-ca35-41d0-a8c7-bdc9ec6a82af,2018.0,28,13,0,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...",{JournalArticle},2018-09-03,"{""name"": ""Communications Biology"", ""pages"": nu...",2023-03-31T01:28:37.897Z,5692596
1,1,23708908,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/6ec7c156...,Drug sensitivity profiling identifies potentia...,"{""{\""authorId\"": \""5699001\"", \""name\"": \""H. K...",OncoTarget,106ecc86-1801-4b36-a05c-d36db69a968c,2017.0,46,24,0,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...",{JournalArticle},2017-10-31,"{""name"": ""Oncotarget"", ""pages"": ""97516 - 97527...",2023-04-05T06:18:15.837Z,5692609
2,2,13232625,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/c8566272...,Men's memory for women's sexual-interest and r...,"{""{\""authorId\"": \""4425040\"", \""name\"": \""T. T...",None,None,2011.0,36,13,0,True,"{""{\""category\"": \""Psychology\"", \""source\"": \...",{Review},2011-09-01,"{""name"": ""Applied Cognitive Psychology"", ""page...",2023-04-05T08:18:02.240Z,5692611
3,3,73484844,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/d4c9b2fa...,The Reactive Plasticity of Hippocampal Ionotro...,"{""{\""authorId\"": \""2225106\"", \""name\"": \""A. M...",International Journal of Molecular Sciences,8506a01a-40b8-4e6f-bbb8-ce2492139c15,2019.0,97,15,1,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...","{Review,JournalArticle}",2019-02-27,"{""name"": ""International Journal of Molecular S...",2023-05-20T17:20:54.230Z,5692615
4,4,229159752,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/7f13abe2...,High Mortality Rate in Adult COVID-19 Inpatien...,"{""{\""authorId\"": \""7685173\"", \""name\"": \""Saee...",Journal of Multidisciplinary Healthcare,12e5e550-28bd-401c-a348-f347956deec4,2020.0,37,17,1,True,"{""{\""category\"": \""Medicine\"", \""source\"": \""s...","{JournalArticle,Review}",2020-12-01,"{""name"": ""Journal of Multidisciplinary Healthc...",2023-05-23T13:34:07.581Z,5692620


In [ ]:
# Metadata
from sqlalchemy import MetaData

md = MetaData(engine)

In [5]:
# 

# Query to pull in corpus IDs

sql = '''
    SELECT * FROM attributes LEFT JOIN abstracts ON (CAST(attributes."corpusid" AS VARCHAR) == CAST(abstracts."corpusId" AS VARCHAR));
'''

# Context manager to open the connection to the database and execute the query. 
with engine.connect() as conn:
    query = conn.execute(text(sql))
    
# Get all records and store in dataframe
# df = pd.DataFrame(query.fetchall())
# df.head()

ProgrammingError: (psycopg2.errors.UndefinedFunction) operator does not exist: character varying == character varying
LINE 2: ...tracts ON (CAST(attributes."corpusid" AS VARCHAR) == CAST(ab...
                                                             ^
HINT:  No operator matches the given name and argument types. You might need to add explicit type casts.

[SQL: 
    SELECT * FROM attributes LEFT JOIN abstracts ON (CAST(attributes."corpusid" AS VARCHAR) == CAST(abstracts."corpusId" AS VARCHAR));
]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [5]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [14]:
metadata = MetaData(bind=engine)

In [16]:
metadata.reflect(engine)

In [17]:
from sqlalchemy import inspect
inspector = inspect(engine)
schemas = inspector.get_schema_names()

for schema in schemas:
    print("schema: %s" % schema)
    for table_name in inspector.get_table_names(schema=schema):
        for column in inspector.get_columns(table_name, schema=schema):
            print("Column: %s" % column)

schema: information_schema
Column: {'name': 'feature_id', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
Column: {'name': 'feature_name', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
Column: {'name': 'is_supported', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
Column: {'name': 'is_verified_by', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
Column: {'name': 'comments', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
Column: {'name': 'implementation_info_id', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
Column: {'name': 'implementation_info_name', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
Column: {'name': 'integer_value', 'type': INTEGER(), 'nulla